## Tabla de Contenidos

1. <a href='#section_Importacion_de_modules'>Importación de modules</a>
2. <a href='#section_Importacion_de_dataset'>Importación del dataset patient_covid.csv</a>
3. <a href='#section_Modelos'>Modelos</a> 
</br>3.1 <a href='#section_KNN'>KNN</a>
</br>3.2 <a href='#section_rl'>Regresión logística</a>
</br>3.3 <a href='#section_nbm'>Naive Bayes multinomial</a>
4. <a href='#section_1'>otro1</a> 

------------
</br>3.1.1. <a href='#section_preparacion'>Preparar la matriz de _features_ y el vector _target_</a>
</br>3.1.1. <a href='#section_optimizando'>Optimizando el valor de _k_</a>
</br>3.1.2. <a href="#section_confusion">Matriz de confusión</a>
</br>3.1.3. <a href="#section_conclusion">Conclusión</a>


# **1. Importación de *modules***
<a id="section_Importacion_de_modules"></a>

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn import metrics
from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_score, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score ,f1_score ,classification_report , confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize, StandardScaler

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings("ignore")

import funciones as f

# **2. Importación del dataset patient_covid.csv**
<a id="section_Importacion_de_dataset"></a>

In [2]:
data_covid = pd.read_csv("../data/patient_covid.csv", sep = ",", low_memory=False) 
f._get_info(data_covid)

#data_covid_knn = pd.read_csv("../data/patient_covid_knn.csv", sep = ",", low_memory=False) 
data_covid_knn = pd.read_csv("../data/patient_covid.csv", sep = ",", low_memory=False) 
f._get_info(data_covid_knn)

   inpatient  agegroup_10-19  agegroup_20-29  agegroup_30-39  agegroup_40-49  \
0          0               0               0               0               1   
1          0               0               0               0               0   
2          1               0               0               0               0   

   agegroup_50-59  agegroup_60-69  agegroup_70-79  agegroup_80-89  \
0               0               0               0               0   
1               1               0               0               0   
2               1               0               0               0   

   agegroup_90-99  ...  diabetes_Y  hypertension_Y  immunosuppression_Y  \
0               0  ...           0               0                    0   
1               0  ...           0               0                    0   
2               0  ...           1               1                    0   

   obesity_Y  other_diseases_N  other_diseases_Y  pneumonia_Y  pregnant_Y  \
0          0            

# **3.Modelos**
<a id="section_Modelos"></a>

## **3.1. KNN** <a id="section_KNN"></a>

#### ***Preparar la matriz de _features_ y el vector _target_***
<a id="section_preparacion"></a>


In [3]:
data_covid_knn['inpatient'].value_counts(normalize=True)

0    0.734629
1    0.265371
Name: inpatient, dtype: float64

In [4]:
#cols_features=data_covid_knn.columns.difference(['inpatient','agegroup'])
cols_features=data_covid_knn.columns.difference(['inpatient'])
X = data_covid_knn[cols_features]
y = data_covid_knn['inpatient']
X.head(3) 
X.columns

,agegroup_10-19,agegroup_20-29,agegroup_30-39,agegroup_40-49,agegroup_50-59,agegroup_60-69,agegroup_70-79,agegroup_80-89,agegroup_90-99,asthma_Y,...,diabetes_Y,hypertension_Y,immunosuppression_Y,obesity_Y,other_diseases_N,other_diseases_Y,pneumonia_Y,pregnant_Y,sex_M,smoker_Y
0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,1,1,0,1,1,0,0,0,1,0


Index(['agegroup_10-19', 'agegroup_20-29', 'agegroup_30-39', 'agegroup_40-49',
       'agegroup_50-59', 'agegroup_60-69', 'agegroup_70-79', 'agegroup_80-89',
       'agegroup_90-99', 'asthma_Y', 'cardiovascular_Y',
       'chronic_kidney_failure_N', 'chronic_kidney_failure_Y', 'copd_Y',
       'covid_NOA', 'covid_Y', 'diabetes_N', 'diabetes_Y', 'hypertension_Y',
       'immunosuppression_Y', 'obesity_Y', 'other_diseases_N',
       'other_diseases_Y', 'pneumonia_Y', 'pregnant_Y', 'sex_M', 'smoker_Y'],
      dtype='object')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,stratify=data_covid['inpatient'],random_state=123)
display(y_train.value_counts(normalize=True).round(2))
display(y_test.value_counts(normalize=True).round(2))

0    0.73
1    0.27
Name: inpatient, dtype: float64

0    0.73
1    0.27
Name: inpatient, dtype: float64

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

#### ***Modelo Base***

In [7]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred).round(2)

KNeighborsClassifier()

0.86

#### ***Optimizando el valor de k***

<font color='red'>cambio temporario de range(10, 31) a range(10, 21)</font>

In [ ]:
# Definimos la estrategia de validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=12)
scores_para_df_standard = []

#for i in range(10, 31):
for i in range(18, 21):
    model = KNeighborsClassifier(n_neighbors=i)
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf)
    dict_row_score = {'score_medio':np.mean(cv_scores),
                      'score_std':np.std(cv_scores), 'n_neighbors':i}
    scores_para_df_standard.append(dict_row_score)

KeyboardInterrupt: 

In [ ]:
# Creamos el DataFrame a partir de la lista de diccionarios
df_scores_standard = pd.DataFrame(scores_para_df_standard)
df_scores_standard.head()

Graficando la búsqueda del mejor hiperparámetro

In [ ]:
# Generamos los límites inferior y superior
df_scores_standard['limite_superior'] = df_scores_standard['score_medio'] + df_scores_standard['score_std']
df_scores_standard['limite_inferior'] = df_scores_standard['score_medio'] - df_scores_standard['score_std']
df_scores_standard.head()

In [ ]:
# Graficamos los resultados
plt.plot(df_scores_standard['n_neighbors'], df_scores_standard['limite_inferior'], color='r')
plt.plot(df_scores_standard['n_neighbors'], df_scores_standard['score_medio'], color='b')
plt.plot(df_scores_standard['n_neighbors'], df_scores_standard['limite_superior'], color='r');

In [ ]:
# Identificamos el score máximo
df_scores_standard.loc[df_scores_standard.score_medio == df_scores_standard.score_medio.max()]

In [ ]:
# Asignamos el valor del k óptimo a una variable
best_k = df_scores_standard.loc[df_scores_standard.score_medio == df_scores_standard.score_medio.max(), 'n_neighbors'].values[0]
print("best k",best_k)

# Elegimos el modelo óptimo de acuerdo a las pruebas de cross validation
model = KNeighborsClassifier(n_neighbors=best_k)

# Lo ajustamos sobre los datos de entrenamiento
model.fit(X_train, y_train)

# Evaluamos qué accuracy obtenemos en train
print("accuracy_score en train",accuracy_score(y_train, model.predict(X_train)).round(4))

In [ ]:
# Lo utilizamos para predecir en test
X_test = scaler.transform(X_test) # ¡Importantísimo estandarizar también los datos de test con las medias y desvíos aprendidos en train!
y_pred = model.predict(X_test)

# Evaluamos el accuracy del modelo en test
print("accuracy_score en test",accuracy_score(y_test, y_pred).round(2))



<a id="section_confusion"></a>
## Matriz de confusión

In [ ]:
    plt.rcParams['figure.figsize'] = (10, 5)
    plt.rcParams['font.size'] = 10

In [ ]:
# Obtenemos la matriz de confusión
cm = confusion_matrix(y_test, y_pred)
cm_print_matriz_confusion(y_test, y_pred)

#### **Graficar la matriz de confusión**

<img src="http://acnalert.eastus.cloudapp.azure.com/2/ds/Confusion.jpg" alt="Matriz confusion">

In [ ]:
#  calcular  la exactitud de nuestro modelo...
print("Acuracy score KNN :\n", accuracy_score(y_test, y_pred).round(2))
print("Classification report KNN:\n", classification_report(y_test,y_pred))

In [ ]:
probs=lr.predict_proba(x_test)
preds=probs[:,1]
fpr,tpr, threshold_lr =metrics.roc_curve(y_test, y_pred)
roc_auc_lr=metrics.auc(fpr_lr,tpr_lr)

#graph
plt.title("ROC")

plt.plot(fpr, tpr, 'b', label="KNN" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0,1],[0,1],'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

<a id="section_conclusion"></a>
## Conclusión

## **3.2. Regresión logística** <a id="section_rl"></a>

In [ ]:
print(data_covid.nunique())
print(data_covid.shape)

In [ ]:
x=data_covid.drop(["inpatient"],axis=1)
y=data_covid["inpatient"]
y=pd.DataFrame(y)
y

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,stratify=y)
print ("X_train:" , x_train.shape[0])
print ("y_train:" , y_train.shape[0])
print ("X_test:" , x_test.shape[0])
print ("y_test:" , y_test.shape[0])

In [ ]:
y_train=pd.DataFrame(y_train)
y_train

In [ ]:
#Logistic regression
#scaler = StandardScaler()
#x_trainstd = scaler.fit_transform(x_train)
#x_teststd = scaler.transform(x_test)
lr = LogisticRegression(class_weight="balance")
lr.fit(x_train, y_train)
score_lr= lr.score(x_test, y_test)

In [ ]:
print ("Logistic Regression Score:", score_lr)

In [ ]:
y_pred_lr = lr.predict(x_test)

In [ ]:
print("AS LR:\n", accuracy_score(y_test, y_pred_lr))

In [ ]:
print("CM LR:\n", confusion_matrix(y_test, y_pred_lr))

In [ ]:
lr.intercept_

In [ ]:
lr.coef_

In [ ]:
print("LR:\n", classification_report(y_test,y_pred_lr))

In [ ]:
importance = lr.coef_.flatten()

In [ ]:
plt.rcParams["figure.figsize"] =(10,10)
plt.barh(x.columns,importance,color="g")
plt.title("Barplot Summary of Feature Importance")
plt.xlabel("score")
plt.show()

In [ ]:
#LR
probs_lr=lr.predict_proba(x_test)
preds_lr=probs_lr[:,1]
fpr_lr,tpr_lr, threshold_lr =metrics.roc_curve(y_test, y_pred_lr)
roc_auc_lr=metrics.auc(fpr_lr,tpr_lr)

In [ ]:
plt.title("ROC")

plt.plot(fpr_lr, tpr_lr, 'b', label="LR" % roc_auc_lr)

plt.legend(loc="lower right")
plt.plot([0,1],[0,1],'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### 4.2.1. Búsqueda y selección del hiperparámetro

##  **3.3. Naive Bayes multinomial**  <a id="section_nbm"></a>

In [ ]:
## 4.1.1. Categorización de edad (o no :/)

#  **4.  otro <a id="section_1"></a>**